In [1]:

from pytelem.optimus import *
from jax import jit

/home/saji/Documents/Code/buildroot/gotelem/py/pytelem/optimus.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/saji/Documents/Code/buildroot/gotelem/py/pytelem/optimus.py:65: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/saji/Documents/Code/buildroot/gotelem/py/pytelem/optimus.py:79: NumbaDeprecationWarning: The 'nopython' keyword argu

In [2]:
ffast = jit(optim.solar_position)

NameError: name 'optim' is not defined

In [6]:
timestamp = np.array(1687306901)
timestamps = np.array([1687306901, 1687306902, 1687306903, 1687306906])

In [3]:

jd = timestamps / 86400.0 + 2440587.5
jde = jd + DELTA_T / 86400.0
jce = (jde - 2451545) / 36525

jme = jce / 10


In [4]:

# todo: make more elegant?
# TODO: vectorize later? it's kinda complex

# heliocentric longitude
l_rad = np.zeros_like(timestamp)
for idx, vec in enumerate(HELIO_L):
    l_rad = l_rad + helio_vector(vec, jme) * jme ** idx

l_rad = l_rad / 10e8
l_deg = np.rad2deg(l_rad) % 360

# heliocentric latitude
b_rad = np.zeros_like(timestamp)
for idx, vec in enumerate(HELIO_B):
    b_rad = b_rad + helio_vector(vec, jme) * jme ** idx
b_rad = b_rad / 10e8
b_deg = b_rad % 360

# heliocentric radius
r_rad = np.zeros_like(timestamp)
for idx, vec in enumerate(HELIO_R):
    r_rad = r_rad + helio_vector(vec, jme) * jme ** idx
r_rad = r_rad / 10e8
r_deg = r_rad % 360

theta = (l_deg + 180) % 360
beta = -1 * b_deg



In [11]:

def cubic_poly(a,b,c,d):
    return a + b * jce + c * jce ** 2 + (jce ** 3) / d
X0 = cubic_poly(297.85036, 445267.111480, -0.0019142, 189474)
X1 = cubic_poly(357.52772, 35999.050340, -0.0001603, -300000)
X2 = cubic_poly(134.96298, 477198.867398, 0.0086972, 56250)
X3 = cubic_poly(93.27191, 483202.017538, -0.0036825, 327270)
X4 = cubic_poly(125.04452, 1934.136261, 0.0020708, 450000)

X = np.vstack([X0, X1, X2, X3, X4]).T
print(X.shape)


(4, 5)


In [13]:

nut = NUTATION_ABCD_ARRAY


In [14]:
(nut[:,0] + jce[..., np.newaxis] * nut[:,1])

(4, 63)

In [25]:

print(X.shape)
print(NUTATION_YTERM_ARRAY.shape)
d_psi = (nut[:,0] + jce[..., np.newaxis] * nut[:,1]) * np.sin(np.sum(X[:, np.newaxis, :] * NUTATION_YTERM_ARRAY[np.newaxis, ...], axis=-1))

print(f"d psi shape {d_psi.shape}")

nut_long = np.sum(d_psi, axis=-1) / 36,000,000

print(nut_long)

(4, 5)
(63, 5)
d psi shape (4, 63)
[-0.00333032 -0.00333032 -0.00333032 -0.00333032]


In [12]:
u = timestamps[:, np.newaxis] ** np.arange(0,11)
print(u.shape)

(4, 11)


In [14]:

epsilon_0 = np.array(
        [[
            84381.448,
            -4680.93,
            1.55,
            1999.25,
            -51.38,
            -249.67,
            -39.05,
            7.12,
            27.87,
            5.79,
            2.45,
        ]]
    )
res = u * epsilon_0


In [16]:
res.sum(axis=-1)

Array([-8.6115017e+12, -5.7323689e+12, -6.4693841e+12, -1.1346544e+13],      dtype=float32)